# ZeissReader Demo

## Data format: .txrm file

## CIL Version

This notebook was developed using CIL v25.0.0

## Dataset
The data is available from: https://zenodo.org/records/14993402

Update this filepath to where you have saved the dataset:

In [ ]:
file_path = '../Data/bin/seed_res2_bin.txrm'

## Loading Geometry

In [ ]:
from cil.io import ZEISSDataReader

from cil.utilities.display import show2D, show_geometry

from cil.framework.labels import AcquisitionDimension

In [ ]:
reader = ZEISSDataReader(file_name=file_path)

In [ ]:
acq_geom = reader.get_geometry()

show_geometry(acq_geom);

In [ ]:
print(acq_geom)

## Loading Projections


In [ ]:
data = reader.read()
data_copy = reader.read()
show2D(data, cmap='inferno');

In [ ]:
import numpy as np
#altered the projection angles to correct the projection geometry, ensuring the reconstruction lines up correctly instead of appearing rotated.
data_copy.geometry.set_angles(data.geometry.angles, initial_angle=-45*np.pi/180, angle_unit='radian')

# Pre-processing and Reconstruction

In [ ]:
from cil.processors import CentreOfRotationCorrector, TransmissionAbsorptionConverter
from cil.recon import FDK

data_exp = TransmissionAbsorptionConverter()(data, out=data)

data_cor = CentreOfRotationCorrector.image_sharpness(backend="tigre")
data_cor.set_input(data_exp)

corrected_data = data_cor.get_output()

recon = FDK(corrected_data)
recon = recon.run()

In [ ]:
show2D(recon, title='Reconstruction FDK',  origin='upper-right', cmap='inferno', fix_range=(-0.3, 1.5), slice_list=[524]);

In [ ]:
data_exp = TransmissionAbsorptionConverter()(data_copy, out=data_copy)

data_cor = CentreOfRotationCorrector.image_sharpness(backend="tigre")
data_cor.set_input(data_exp)

corrected_data = data_cor.get_output()

#defining img geometry and reducing size
ig = corrected_data.geometry.get_ImageGeometry()
ig.voxel_num_y = 500

rotated_recon = FDK(corrected_data, image_geometry=ig)
rotated_recon = rotated_recon.run()

In [ ]:
from cil.utilities.jupyter import islicer

islicer(data, cmap='inferno')


In [ ]:
islicer(recon, cmap='inferno')

In [ ]:
recon_list = [recon, rotated_recon]
recon_labels = ['Original Reconstruction', 'Rotated Reconstruction']
show2D(recon_list, title=recon_labels, cmap='inferno', num_cols=2, fix_range=(-0.3, 1.5), slice_list=[524,524])